In [89]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_json,col
from pyspark.sql.types import *
from os.path import abspath

spark = SparkSession\
        .builder\
        .appName("pyspark-notebook")\
        .master("spark://spark-master:7077")\
        .config("spark.executor.memory", "512m")\
        .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
        .enableHiveSupport()\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [4]:
!pip install opendatasets

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [5]:
import opendatasets as od
from pathlib import Path

In [7]:
dataset_path = Path('new-york-city-airbnb-open-data', 'AB_NYC_2019.csv')
if not dataset_path.is_file():
    od.download('https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  nata44845


Your Kaggle Key:

  ································


100%|██████████| 2.44M/2.44M [00:01<00:00, 2.46MB/s]

In [90]:
data = spark.read.csv(path="new-york-city-airbnb-open-data/AB_NYC_2019.csv", sep=",", header=True)

In [91]:
data.count()

49079

In [92]:
data.head()

Row(id='2539', name='Clean & quiet apt home by the park', host_id='2787', host_name='John', neighbourhood_group='Brooklyn', neighbourhood='Kensington', latitude='40.64749', longitude='-73.97237', room_type='Private room', price='149', minimum_nights='1', number_of_reviews='9', last_review='2018-10-19', reviews_per_month='0.21', calculated_host_listings_count='6', availability_365='365')

In [93]:
df1 = data.select(["price"])
df = df1.filter("price is not NULL and price!='Private room' and price!='Entire home/apt' and price!='Manhattan'")
df.head()

Row(price='149')

In [75]:
df.write.saveAsTable('AB_NYC_2019_price1')

In [94]:
ndf=spark.sql("""select count(*) CNT from AB_NYC_2019_price1""")

In [95]:
ndf.show()

+-----+
|  CNT|
+-----+
|48887|
+-----+



In [97]:
ndf=spark.sql("""select AVG(price) M,SQRT(VAR_POP(price)) D,MIN(price) minp, MAX(price) maxp from AB_NYC_2019_price1""")

In [98]:
ndf.show()

+------------------+------------------+---------+----+
|                 M|                 D|     minp|maxp|
+------------------+------------------+---------+----+
|152.22296299343384|238.53904666725555|-73.99986|9999|
+------------------+------------------+---------+----+



In [100]:
df.head()
list_values = [float(row.price) for row in df.collect()]
print(len(list_values))

48887


In [101]:
n, mean, M2 = 0, 0.0, 0
for score in list_values:
    n += 1
    delta = score - mean
    mean += delta / n
    M2 += delta * (score - mean)
print(n, mean, (M2 / n) ** (1/2))

48887 152.22296299343435 238.53904666725515
